In [10]:
# from getopt import getopt
# import cloudpickle
# import pickle
# import sys
# import os
import numpy as np
import pandas as pd
import seaborn as sns
# from scipy.special import logit, expit
from scipy.stats import uniform, norm, bernoulli
# from statsmodels.stats.proportion import proportions_ztest
from matplotlib import pyplot as plt
# import pymc as pm
# import arviz as az
# from modeltools import mcmc_diagnostics, create_summary_stat
# from downcast import downcast_df
# import jax
# from pymc.sampling_jax import sample_numpyro_nuts
# from time import time, sleep
# from datetime import timedelta

In [12]:
test = pd.read_csv("data/test/score_0.csv")
testOA = pd.read_csv("data/testOA/score_0.csv")

In [9]:
test["sim_rater_id"].nunique()

79

In [17]:
test.groupby("sim_topic_id").agg({"trial_id":"count"})

,trial_id
sim_topic_id,
0,19
1,19
2,19
3,19
4,19
...,...
95,19
96,19
97,19


In [18]:
testOA.groupby("sim_topic_id").agg({"trial_id":"count"})

,trial_id
sim_topic_id,
0,15
1,24
2,28
3,23
4,18
...,...
95,28
96,19
97,23


In [21]:
for c in testOA:
    testOA=testOA.rename(columns={c:c+"OA"})

In [24]:
a = pd.merge(test, testOA, left_on=["sim_topic_id", "sim_rater_id"],
            right_on=["sim_topic_idOA", "sim_rater_idOA"], how="inner")

In [26]:
sum(a["intrusion"]==a["intrusionOA"])

704